In [1]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [2]:
using Pkg

pkg"add Embeddings; precompile;"
using Embeddings

pkg"add Distances; precompile;"
using Distances

const w2v = load_embeddings(Word2Vec)

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed AutoHashEquals ─ v0.2.0
 Installed Reexport ─────── v0.2.0
 Installed IniFile ──────── v0.5.0
 Installed Embeddings ───── v0.3.1
 Installed DataDeps ─────── v0.7.0
 Installed HTTP ─────────── v0.8.7
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5bfea45] + Embeddings v0.3.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [15f4f7f2] + AutoHashEquals v0.2.0
  [124859b0] + DataDeps v0.7.0
  [c5bfea45] + Embeddings v0.3.1
  [cd3eb016] + HTTP v0.8.7
  [83e8ac13] + IniFile v0.5.0
  [189a3867] + Reexport v0.2.0
Precompiling project...
Precompiling Embeddings


┌ Info: Precompiling Embeddings [c5bfea45-b7f1-5224-a596-15500f5db411]
└ @ Base loading.jl:1192


 Resolving package versions...
 Installed Distances ─ v0.8.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [b4f34e82] + Distances v0.8.2
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [b4f34e82] + Distances v0.8.2
Precompiling project...
Precompiling Distances


┌ Info: Precompiling Distances [b4f34e82-e78d-54a5-968a-f98e89d6e8f7]
└ @ Base loading.jl:1192


This program has requested access to the data dependency word2vec 300d.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Pretrained Word2Vec Word emeddings
Website: https://code.google.com/archive/p/word2vec/
Author: Mikolov et al.
Year: 2013

Pre-trained vectors trained on part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases.

Paper:
    Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, and Jeffrey Dean. Distributed Representations of Words and Phrases and their Compositionality. In Proceedings of NIPS, 2013.



Do you want to download the dataset from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz to "/root/.julia/datadeps/word2vec 300d"?
[y/n]
stdin> y


┌ Info: Downloading
│   source = https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
│   dest = /root/.julia/datadeps/word2vec 300d/GoogleNews-vectors-negative300.bin.gz
│   progress = 0.1036
│   time_taken = 5.0 s
│   time_remaining = 43.25 s
│   average_speed = 32.555 MiB/s
│   downloaded = 162.807 MiB
│   remaining = 1.375 GiB
│   total = 1.534 GiB
└ @ HTTP /root/.julia/packages/HTTP/6Smhf/src/download.jl:119
┌ Info: Downloading
│   source = https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
│   dest = /root/.julia/datadeps/word2vec 300d/GoogleNews-vectors-negative300.bin.gz
│   progress = 0.2165
│   time_taken = 10.0 s
│   time_remaining = 36.19 s
│   average_speed = 34.005 MiB/s
│   downloaded = 340.119 MiB
│   remaining = 1.202 GiB
│   total = 1.534 GiB
└ @ HTTP /root/.julia/packages/HTTP/6Smhf/src/download.jl:119
┌ Info: Downloading
│   source = https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bi

Embeddings.EmbeddingTable{Array{Float32,2},Array{String,1}}(Float32[0.0673199 0.0529562 … -0.21143 0.0136373; -0.0534466 0.0654598 … -0.0087888 -0.0742876; … ; -0.00733469 0.0108946 … -0.00405157 0.0156112; -0.00514565 -0.0470722 … -0.0341579 0.0396559], ["</s>", "in", "for", "that", "is", "on", "##", "The", "with", "said"  …  "#-###-PA-PARKS", "Lackmeyer", "PERVEZ", "KUNDI", "Budhadeb", "Nautsch", "Antuane", "tricorne", "VISIONPAD", "RAFFAELE"])

# Embeddings #

Up until this point we have dealt with continuous data (images, measurements, etc.), but a healthy amount of data one can be interested in is discrete. Think of graph structures, language, etc. These pose a challenge for neural models, why?

The vocabulary of a model for language is frequently in the millions and a big knowledge graph like Wikidata contains [66,689,671](https://www.wikidata.org/wiki/Wikidata:Statistics) entities.

What does a good representation look like in an abstract sense?:

1. Representations are **distinct**

2. Similar items have **similar** representations

If both of these statements hold, it is likely that a representation can be useful for a downstream machine learning system.

## An indicator function ##

For more “traditional” machine learning systems, we are likely to use an indicator function such as:

$$f_{id}(w) \mapsto \mathbb{N^{*}}$$

We can then create a sparse binary representation using this function:

$$g(w, i) = {\left\lbrace
                            \begin{array}{ll}
                                1 & \textrm{if }~i = f_{id}(w) \\
                                0 & \textrm{otherwise} \\
                            \end{array}\right.}$$

$$f_{sb}(w) = [g(w, 1), \ldots]$$


As a concrete “toy” example, we can imagine:

$$\mathbb{V} = \{\textrm{cat}, \textrm{dog}, \textrm{human}\}$$

$$ f_{id}(\textrm{cat}) = 1, \ldots, f_{id}(\textrm{human}) = 3$$

$$f_{sb}(\textrm{cat}) = [1, 0, 0]$$

$$f_{sb}(\textrm{dog}) = [0, 1, 0]$$

$$f_{sb}(\textrm{human}) = [0, 1, 0]$$

A big problem with this approach is that there's no distinction between how similar things are. I.e. everything is orthogonal and cat/dog/human are all equally similar / different

## A case study: word2vec ##

**Cbow representation**: fill in the blanked word based on the window around it

**Skip-gram**: predict all words in the window based on $w(t)$

![](https://cdn-images-1.medium.com/max/1600/1*GkL9XH1XXSUU0RfAHKJGCg.png)

[Image source](https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf)

“If A and B have almost identical environments we say that they are synonyms” – Zellig Harris (1954)

“You shall know a word by the company it keeps” – John Rupert Firth (1957)

For some intuition, consider: “I had some _____ for breakfast today”, you instinctively know that “airplanes” is likely to be wrong, but “toast” is likely to be right.




### Formalised ###

$$ W^w \in \mathbb{R}^{|\mathbb{V}| \times d} $$

$$ w \in \mathbb{V} $$

$$ W^c \in \mathbb{R}^{|\mathbb{V}| \times d} $$

$$ c \in \mathbb{V} $$

$$ σ(x)  = \frac{1}{1 + e^{-x}} $$

$$ \mathcal{D} = \{(w_1, c_1), \ldots, (w_n, c_n)\} $$

$$ \mathcal{D}^{\prime} = \{(w_1, c_1), \ldots, (w_m, c_m)\} $$

$$ p((c, w) \in \mathbb{D}|c, w, W^{w}, W^{c}) = \sigma(W^{c}_{f_{id}(c), :} \cdot W^{w}_{f_{id}(w), :}) $$

$$ arg\max_{w,b}
    \sum_{(w, c) \in D} \textrm{log}(W^{c}_{f_{id}(c), :} \cdot W^{w}_{f_{id}(w), :})
    + \sum_{(w, c) \in D^{\prime}} \textrm{log}(-W^{c}_{f_{id}(c), :} \cdot W^{c}_{f_{id}(w), :})
$$

Why is $\mathcal{D}^{\prime}$ necessary? What happens if we remove it?

The $\mathcal{D}^{\prime}$ stuff is referred to as *noise contrasted optimisation*

In [3]:
const token2index = Dict(token => i for (i, token) in enumerate(w2v.vocab))
embedding(token)  = w2v.embeddings[:, token2index[token]]

embedding (generic function with 1 method)

### Cosine distance ###

$$ cos(u, v) = 1 - \frac{u \cdot v}{||u|| ||v||} $$

$$ cos(u, v) \mapsto [0, 2] $$

$$ cos(u, v) = 0 \implies \textrm{identical} $$

$$ cos(u, v) = 2 \implies \textrm{opposites} $$

$$ cos(u, v) = 1 \implies \textrm{orthogonal} $$

Why are we not using something more “standard” like the Euclidian distance?

In [4]:
distance(tokena, tokenb) = cosine_dist(embedding(tokena), embedding(tokenb))

@show distance("dog", "cat")
@show distance("cat", "dog") # Symmetric, of course.
@show distance("cat", "human")
@show distance("dog", "human")
@show distance("ape", "human");

distance("dog", "cat") = 0.2390545f0
distance("cat", "dog") = 0.2390545f0
distance("cat", "human") = 0.75563335f0
distance("dog", "human") = 0.8109204f0
distance("ape", "human") = 0.6500703f0


![](https://cdn-images-1.medium.com/max/1600/1*vvtIsW1AblmgLkq1peKfOg.png)

[Image source](https://towardsdatascience.com/mapping-word-embeddings-with-word2vec-99a799dc9695)

## Neighbourhoods ##

In [5]:
function neighbourhood(emb; n=32)
    nneighbours = Int[]
    maxdist     = typemax(Float32)
    for i in eachindex(w2v.vocab)
        d = cosine_dist(emb, w2v.embeddings[:, i])
        if length(nneighbours) > n && d >= maxdist
            continue
        end
        push!(nneighbours, i)
        sort!(nneighbours,
            by=j -> cosine_dist(emb, w2v.embeddings[:, j]))
        maxdist = cosine_dist(emb, w2v.embeddings[:, nneighbours[end]])
        length(nneighbours) <= n || pop!(nneighbours)
    end
    [(w2v.vocab[i], cosine_dist(emb, w2v.embeddings[:, i]))
        for i in nneighbours]
end
neighbourhood(token::String; n=32) = neighbourhood(embedding(token), n=n)

neighbourhood (generic function with 2 methods)

In [6]:
neighbourhood("religion")

32-element Array{Tuple{String,Float32},1}:
 ("religion", 0.0)            
 ("religions", 0.2513935)     
 ("religious", 0.27735394)    
 ("Christianity", 0.29210013) 
 ("Religion", 0.34160268)     
 ("relgion", 0.35627824)      
 ("religon", 0.36557502)      
 ("Islam", 0.37923366)        
 ("faiths", 0.38649082)       
 ("religous", 0.40177977)     
 ("spirituality", 0.40435177) 
 ("religiosity", 0.4094324)   
 ("morality", 0.41415274)     
 ⋮                            
 ("religiousness", 0.43363798)
 ("Hinduism", 0.43809837)     
 ("Buddism", 0.43927014)      
 ("Judaism", 0.44100153)      
 ("teachings", 0.44211966)    
 ("Paganism", 0.44220132)     
 ("Universalism", 0.44323426) 
 ("secularism", 0.44477552)   
 ("Buddhism", 0.44490796)     
 ("nonreligion", 0.4462548)   
 ("creed", 0.44685173)        
 ("ethnicity", 0.4470787)     

In [0]:
neighbourhood("england")

32-element Array{Tuple{String,Float32},1}:
 ("england", 0.0)         
 ("liverpool", 0.28275466)
 ("chelsea", 0.29395294)  
 ("fulham", 0.29792225)   
 ("tottenham", 0.3092957) 
 ("rooney", 0.3136869)    
 ("torres", 0.32362872)   
 ("ronaldo", 0.3242663)   
 ("spain", 0.32593936)    
 ("gerrard", 0.32909578)  
 ("anelka", 0.3319348)    
 ("fergie", 0.3326596)    
 ("europe", 0.335052)     
 ⋮                        
 ("newcastle", 0.3432166) 
 ("leeds", 0.34379578)    
 ("barca", 0.34468734)    
 ("madrid", 0.3449734)    
 ("everton", 0.34627277)  
 ("juve", 0.3468153)      
 ("wenger", 0.34718215)   
 ("benitez", 0.3498447)   
 ("beckham", 0.3502348)   
 ("henry", 0.35096765)    
 ("italy", 0.3517356)     
 ("pompey", 0.35407293)   

### Word representation algebra ###

$$ f_{n}(\textrm{king}) - f_{n}(\textrm{man}) + f_{n}(\textrm{woman}) \approx f_{n}(\textrm{queen}) $$

$$ f_{n}(\textrm{Paris}) - f_{n}(\textrm{France}) + f_{n}(\textrm{Italy}) \approx f_{n}(\textrm{Rome})$$

This is notoriously difficult to replicate though…

In [7]:
isto(ais, tob, likecto) = neighbourhood(embedding(ais) .- embedding(tob) .+ embedding(likecto))

isto (generic function with 1 method)

In [8]:
isto("king", "man", "woman")

32-element Array{Tuple{String,Float32},1}:
 ("king", 0.2007401)         
 ("queen", 0.28818095)       
 ("monarch", 0.3810326)      
 ("princess", 0.4097572)     
 ("prince", 0.462268)        
 ("kings", 0.4763158)        
 ("queens", 0.4818864)       
 ("sultan", 0.4901408)       
 ("monarchy", 0.49125886)    
 ("throne", 0.49941933)      
 ("royal", 0.5061796)        
 ("ruler", 0.50907254)       
 ("empress", 0.5112186)      
 ⋮                           
 ("Mswati", 0.5552143)       
 ("maharaja", 0.55632824)    
 ("princesses", 0.5576925)   
 ("Princess", 0.5624594)     
 ("duchess", 0.5627598)      
 ("Queen", 0.5653622)        
 ("monarchs", 0.5656772)     
 ("handmaid", 0.5685401)     
 ("Gosakuramachi", 0.5699681)
 ("kumari", 0.57787603)      
 ("maharani", 0.57856727)    
 ("commoner", 0.57872516)    

In [0]:
isto("Paris", "France", "Italy")

32-element Array{Tuple{String,Float32},1}:
 ("Milan", 0.27778602)  
 ("Rome", 0.2971689)    
 ("Italy", 0.3267352)   
 ("Paris", 0.3448093)   
 ("Italian", 0.4088726) 
 ("Tuscany", 0.43671834)
 ("Bologna", 0.43916416)
 ("Sicily", 0.44036168) 
 ("Genoa", 0.46910983)  
 ("Milanese", 0.4694308)
 ("Rimini", 0.48378998) 
 ("Venice", 0.4858547)  
 ("ANSA", 0.4889533)    
 ⋮                      
 ("Pisa", 0.499672)     
 ("Liguria", 0.5008726) 
 ("Sassari", 0.50194716)
 ("Brianza", 0.50275654)
 ("Sempione", 0.5056912)
 ("Pietro", 0.5061012)  
 ("Pomezia", 0.50707614)
 ("Giorgio", 0.5097326) 
 ("Vicenza", 0.5102454) 
 ("Gianni", 0.51183736) 
 ("Pistoia", 0.51197916)
 ("Jacopo", 0.51241815) 

![](https://samyzaf.com/ML/nlp/word2vec2.png)

[Image source](https://samyzaf.com/ML/nlp/nlp.html)

Why and how would these relationships be preserved in the embeddings? Think, what do they capture?

**Beware** when playing around with these models though, they are a reflection of the underlying data and very well mirrors the worst of us as a species in our racism, sexism, etc.

## Some closing comments and questions… ##

The same line of thinking case be used for graph embeddings, etc. I also find it helpful to think about what an MLP or CNN achieves in a similar way.

1. How do we handle sequences?

2. Our representations are context independent, is this a problem?

3. Can we imagine loss functions akin to word2vec for other areas?

4. From your Linear Algebra, does $W^w$ and $W^c$ remind you of something? Does this hint at another way to learn embeddings?

## Extracurricular reading ##

1. [“Distributed Representations of Words and Phrases and their Compositionality”](https://papers.nips.cc/paper/5021-distributed-representations-of-words-andphrases) by Mikolov et al. (2013)

2. [“word2vec Explained: deriving Mikolov et al.’s negative-sampling word-embedding method”](https://arxiv.org/abs/1402.3722) by Goldberg and Levy (2014). I *highly* recommend to **read this in parallel with Mikolov et al. (2013)**.